In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None

In [2]:
df_original = pd.read_csv('classified_twitter_accounts.csv')

In [3]:
df_original.columns

Index(['id', 'name', 'screen_name', 'statuses_count', 'followers_count',
       'friends_count', 'favourites_count', 'listed_count', 'lang',
       'default_profile', 'default_profile_image', 'geo_enabled', 'protected',
       'verified', 'classification', 'dataset', 'screen_name_total_len',
       'screen_name_num_len', 'name_total_len', 'name_num_len', 'is_url_null',
       'is_location_null', 'profile_banner_url_null', 'profile_image_url_null',
       'description_null'],
      dtype='object')

In [4]:
df_original.shape

(14368, 25)

In [5]:
# DROP FEATURES LIST

#ALTA CARDINALIDADE
alta_card = ['id','name', 'screen_name']

# ALTA CORRELACAO (IDENTIFICADA NA ANALISE EXPLORATORIA)
correlacao = ['listed_count']

construidas = [
    'screen_name_total_len',  
    'screen_name_num_len',       
    'name_total_len',            
    'name_num_len',              
    'is_url_null',               
    'is_location_null',          
    'profile_banner_url_null',   
    'profile_image_url_null',    
    'description_null'
] 


drop_features = alta_card + correlacao + construidas

# VARIAVEIS DE CONTROLE DO DATASET
drop_features.append('dataset')

df_accounts = df_original.drop(drop_features, axis=1)

In [6]:
# target
df_accounts['target'] = np.where(df_accounts.classification == "bot", 1, 0)
df_accounts.drop('classification', axis=1, inplace=True)

In [7]:
df_accounts.dtypes

statuses_count            int64
followers_count           int64
friends_count             int64
favourites_count          int64
lang                     object
default_profile            bool
default_profile_image      bool
geo_enabled                bool
protected                  bool
verified                   bool
target                    int32
dtype: object

In [8]:
df_accounts.shape

(14368, 11)

In [9]:
df_accounts.describe()

,statuses_count,followers_count,friends_count,favourites_count,target
count,14368.000000,1.436800e+04,14368.000000,14368.000000,14368.000000
mean,5063.190910,8.682774e+02,589.318834,1145.288280,0.758213
std,17493.409898,2.688472e+04,2665.832959,6036.894173,0.428181
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,31.000000,6.000000e+00,42.000000,0.000000,1.000000
50%,63.000000,2.500000e+01,250.000000,0.000000,1.000000
75%,1365.250000,3.320000e+02,538.000000,9.000000,1.000000
max,399555.000000,2.991573e+06,211890.000000,313954.000000,1.000000


# CATEGORIZANDO AS VARIÁVEIS

In [10]:
df = df_accounts.copy()

In [11]:
df['statuses_count'] = pd.cut(df.statuses_count, bins=[-1,31,63,1365,399555], labels=[
     '1 - DE 0 ATE 31'
    ,'2 - DE 32 ATE 63'
    ,'3 - DE 64 ATE 1365'
    ,'4 - ACIMA DE 1365'
] )

In [12]:
df['followers_count'] = pd.cut(df.followers_count, bins=[-1,6,25,332,2991573], labels=[
     '1 - DE 0 ATE 6'
    ,'2 - DE 7 ATE 25'
    ,'3 - DE 26 ATE 332'
    ,'4 - ACIMA DE 332'
] )

In [13]:
df['friends_count'] = pd.cut(df.friends_count, bins=[-1,42,250,538,211890], labels=[
     '1 - DE 0 ATE 42'
    ,'2 - DE 43 ATE 250'
    ,'3 - DE 251 ATE 538'
    ,'4 - ACIMA DE 538'
])

In [14]:
df['favourites_count'] = pd.cut(df.favourites_count, bins=[-1,9,313954], labels=[
     '1 - DE 0 ATE 9'
    ,'2 - ACIMA DE 9'
])

In [15]:
df['lang'] = np.where(df.lang == "en", 'ENGLISH', 'OTHERS')
df['lang'] = df['lang'].astype('category')

In [158]:
# df['screen_name_num_len'] = pd.cut(df.screen_name_num_len, bins=[-1,0,13], labels=[
#      '1 - ZERO'
#     ,'2 - ACIMA DE ZERO'
# ])

In [159]:
# df['name_num_len'] = pd.cut(df.name_num_len, bins=[-1,0,25], labels=[
#      '1 - ZERO'
#     ,'2 - ACIMA DE ZERO'
# ])

In [160]:
# df['name_total_len'] = pd.cut(df.name_total_len, bins=[0,11,13,15,50], labels=[
#      '1 - DE 1 ATE 11'
#     ,'2 - DE 12 ATE 13'
#     ,'3 - DE 14 ATE 15'
#     ,'4 - ACIMA DE 15'
# ])

# Criar Dummies

In [16]:
cat_vars = df.drop('target', axis=1).columns

for var in cat_vars:

    cat_list='var' +'_'+ var
    cat_list = pd.get_dummies(df[var], prefix = var, drop_first=True)
    df=df.join(cat_list)

df.drop(cat_vars, axis = 1, inplace = True)

# Regressão Logística

In [17]:
X = df.drop('target', axis = 1)
y = df.target

## BACKWARD FEATURE REMOVAL
drop_list = [
    'protected_True',
    'verified_True',
    'statuses_count_2 - DE 32 ATE 63',
    'default_profile_image_True'
]

X.drop(drop_list, axis = 1, inplace = True)

In [18]:
import statsmodels.api as sm

X = sm.add_constant(X)
logit_model=sm.Logit(y, X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.089737
         Iterations 10
                                  Results: Logit
Model:                    Logit                  Pseudo R-squared:       0.838    
Dependent Variable:       target                 AIC:                    2604.6890
Date:                     2020-06-30 21:08       BIC:                    2703.1349
No. Observations:         14368                  Log-Likelihood:         -1289.3  
Df Model:                 12                     LL-Null:                -7947.4  
Df Residuals:             14355                  LLR p-value:            0.0000   
Converged:                1.0000                 Scale:                  1.0000   
No. Iterations:           10.0000                                                 
----------------------------------------------------------------------------------
                                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
print('Accuracy no treino: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy no teste: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy no treino: 0.98
Accuracy no teste: 0.97


In [21]:
from sklearn.metrics import confusion_matrix

y_pred =  logreg.predict(X_test)
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[1001   47]
 [  68 3195]]
